In [3]:
import os
from utils import json_load

# get pred data

In [6]:
SOURCE_DIR = os.path.join('data', 'source', 'evaluation_dataset')
print(os.listdir(SOURCE_DIR))
llama_zh_dataset_file_path = os.path.join(SOURCE_DIR, '2_llama_zh_evaluation_dataset.json')
#
dataset = json_load(llama_zh_dataset_file_path)

example = dataset[0]

print('reference_answer')
print(example['reference_answer'])
print('reference_context')
print(example['reference_context'])
print('response')
print(example['response'])

['2_llama_zh_evaluation_dataset.json', '4_gemma_evaluation_dataset.json', '5_json_gemma_evaluation_dataset.json', '3_llama_chat_evaluation_dataset.json', '1_llama_en_evaluation_dataset.json']
load data from: data/source/evaluation_dataset/2_llama_zh_evaluation_dataset.json
reference_answer
{'qid': '1', 'stem': '常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?', 'A': '上星、日月', 'B': '合谷、太衝', 'C': '內關、外關', 'D': '上關、下關'}
reference_context
['1.常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？\n\xa0\nA.上星、日月\nB.合谷、太衝\nC.內關、外關\nD.上關、下關']
response
{'A': '三村台终飯加度。', 'B': '吃条台终飯加度。', 'C': '六南分。三村台终飯加度。', 'D': '一条台终飯加度。', 'ans': None, 'qid': 1, 'stem': '台九気终飯加度泩。吃的“六南分。，是一条有分题不别的宽器。'}


# steup

In [7]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate
from llama_index.core.evaluation import CorrectnessEvaluator

llm = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False)

# init

In [8]:
# init
correct_evaluator = CorrectnessEvaluator(llm = llm, score_threshold=4.0)

In [62]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

system_prompt = (
    "你是一個專業的問答聊天機器人評估系統。\n"
    "你會被提供以下資訊：\n"
    "- 使用者的問題 (user query)\n"
    "- 系統生成的答案 (generated answer)\n"
    "同時，你也會被提供一個參考答案 (reference answer)，作為評估的參考依據。\n"
    "你的工作是判斷生成答案的「相關性」與「正確性」。\n"
    "請輸出一個單一分數，代表整體的評估結果。\n"
    "你必須只在第一行輸出分數，不要使用任何其他格式。\n"
    "接著，在第二行解釋你為什麼會給這個分數。\n"
    "評分準則如下：\n"
    "- 分數需介於 1 到 5 之間，1 代表最差，5 代表最好。\n"
    "- 如果生成的答案與使用者的問題無關，請給分數 1。\n"
    "- 如果生成的答案有關聯，但包含錯誤，請給 2 至 3 分。\n"
    "- 如果生成的答案相關且完全正確，請給 4 至 5 分。\n"
    "範例回覆：\n"
    "4.0\n"
    "生成的答案與參考答案在指標上完全相同，但不如參考答案簡潔。"
)

user_prompt = (
    "## 使用者問題\n"
    "{query}\n"
    "## 參考答案\n"
    "{reference_answer}\n"
    "## 系統生成的答案\n"
    "{generated_answer}"
)


chat_template = ChatPromptTemplate(
    message_templates=[
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=system_prompt),
        ChatMessage(
            role=MessageRole.USER, 
            content=user_prompt)])

In [49]:
correct_evaluator?

Type:        CorrectnessEvaluator
String form: <llama_index.core.evaluation.correctness.CorrectnessEvaluator object at 0x7f03a53b17f0>
File:        ~/miniconda3/envs/rag30/lib/python3.12/site-packages/llama_index/core/evaluation/correctness.py
Docstring:  
Correctness evaluator.

Evaluates the correctness of a question answering system.
This evaluator depends on `reference` answer to be provided, in addition to the
query string and response string.

It outputs a score between 1 and 5, where 1 is the worst and 5 is the best,
along with a reasoning for the score.
Passing is defined as a score greater than or equal to the given threshold.

Args:
    eval_template (Optional[Union[BasePromptTemplate, str]]):
        Template for the evaluation prompt.
    score_threshold (float): Numerical threshold for passing the evaluation,
        defaults to 4.0.

In [50]:
# prompt init
correct_evaluator = CorrectnessEvaluator(llm = llm, score_threshold=4.0, eval_template=chat_template)

In [51]:
correct_evaluator.get_prompts()

{'eval_template': ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query', 'reference_answer', 'generated_answer'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='\n你是一個專業的問答聊天機器人評估系統。\n\n你會被提供以下資訊：\n- 使用者的問題 (user query)\n- 系統生成的答案 (generated answer)\n\n同時，你也可能會被提供一個參考答案 (reference answer)，作為評估的參考依據。\n\n你的工作是判斷生成答案的「相關性」與「正確性」。\n\n請輸出一個單一分數，代表整體的評估結果。\n你必須只在第一行輸出分數，不要使用任何其他格式。\n接著，在第二行解釋你為什麼會給這個分數。\n\n評分準則如下：\n- 分數需介於 1 到 5 之間，1 代表最差，5 代表最好。\n- 如果生成的答案與使用者的問題無關，請給分數 1。\n- 如果生成的答案有關聯，但包含錯誤，請給 2 至 3 分。\n- 如果生成的答案相關且完全正確，請給 4 至 5 分。\n\n範例回覆：\n4.0\n生成的答案與參考答案在指標上完全相同，但不如參考答案簡潔。\n')]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='## 使用者問題\n{query}\n\n## 參考答案\n{reference_answer}\n\n## 系統生成的答案\n{generated_answer}\n')])]

# prompt_zh_llama

In [52]:
from llama_index.core.prompts import PromptTemplate

prompt_zh_llama = PromptTemplate(
    "從以下文字中擷取一題選擇題 (MCQ)。如果原始文字沒有提供答案，則完全省略答案欄位，且不要嘗試推測答案\n\n-----\n{text}\n-----\n"
)

# get query

In [53]:
query = prompt_zh_llama.format(text=example['reference_context'][0])
print(query)

從以下文字中擷取一題選擇題 (MCQ)。如果原始文字沒有提供答案，則完全省略答案欄位，且不要嘗試推測答案

-----
1.常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？
 
A.上星、日月
B.合谷、太衝
C.內關、外關
D.上關、下關
-----



# invoke

In [54]:
result = correct_evaluator.evaluate(
    query=query,
    response=str(example['response']),
    reference=str(example['reference_answer']),
)
print(f"feedback: {result.feedback}")
print(f"score: {result.score}")
print(f"passing: {result.passing}")

feedback: 生成答案與題目完全不相干且內容為亂碼/無意義文字，未正確擷取原始題幹與選項（選項與題目不符且被破壞），雖包含 qid 與 ans: None，但未依要求呈現正確的選項欄位與省略答案欄位，因此判定為最差。
score: 1.0
passing: False


In [19]:
type(result.score)

float

In [20]:
type(result.passing)

bool